In [ ]:
from funkcije import *
import matplotlib.pyplot as plt
import pandas as pd
import folium
from folium.plugins import HeatMap
import geopandas as gpd
import contextily as ctx
from pyproj import Transformer

# Predstavitev podatkov

Podatki v CSV datoteki vsebujejo podrobne informacije o prometnih nesrečah in udeležencih v teh nesrečah. Tukaj je opis posameznih stolpcev:

### Osnovni podatki o nesreči
1. **ZaporednaStevilkaPN**: Unikatna številka, ki identificira vsako prometno nesrečo  
2. **KlasifikacijaNesrece**: Klasifikacija nesreče glede na resnost (avtomatično določena glede na najhujšo posledico)  
3. **UpravnaEnotaStoritve**: Upravna enota, kjer se je nesreča zgodila  
4. **DatumPN**: Datum nesreče v formatu dd.mm.llll  
5. **UraPN**: Ura nesreče v formatu hh.mm  

### Lokacijski podatki
6. **VNaselju**: Indikator, ali se je nesreča zgodila v naselju (D) ali izven (N)  
7. **Lokacija**: Lokacija nesreče  
8. **VrstaCesteNaselja**: Vrsta ceste ali naselja, kjer je prišlo do nesreče  
9. **SifraCesteNaselja**: Šifra ceste ali naselja  
10. **TekstCesteNaselja**: Ime ceste ali naselja  
11. **SifraOdsekaUlice**: Šifra odseka ceste ali ulice  
12. **TekstOdsekaUlice**: Ime odseka ceste ali ulice  
13. **StacionazaDogodka**: Točna stacionaža ali hišna številka  
14. **OpisKraja**: Opis prizorišča nesreče  

### Podatki o okoliščinah nesreče
15. **VzrokNesrece**: Glavni vzrok nesreče  
16. **TipNesrece**: Tip nesreče (npr. čelni trk, stranski trk)  
17. **VremenskeOkoliscine**: Vremenske razmere v času nesreče  
18. **StanjePrometa**: Stanje prometa v času nesreče  
19. **StanjeVozisca**: Stanje vozišča v času nesreče  
20. **VrstaVozisca**: Vrsta površine vozišča  
21. **GeoKoordinataX**: X koordinata lokacije (od 2013 v D96/TM, prej Gauß-Krüger)  
22. **GeoKoordinataY**: Y koordinata lokacije (od 2013 v D96/TM, prej Gauß-Krüger)  

### Podatki o udeležencih
23. **ZaporednaStevilkaOsebeVPN**: številka za štetje in ločevanje oseb, udeleženih v prometnih nesrečah
24. **Povzrocitelj**: kot kaj nastopa oseba v prometni nesreči
25. **Starost**: Starost osebe v letih  
26. **Spol**: Spol osebe  
27. **UEStalnegaPrebivalisca**: Upravna enota stalnega prebivališča  
28. **Drzavljanstvo**: Državljanstvo osebe  
29. **PoskodbaUdelezenca**: Stopnja poškodbe udeleženca  
30. **VrstaUdelezenca**: Vloga v prometu (voznik, pešec, kolesar...)  
31. **UporabaVarnostnegaPasu**: Ali je uporabljal varnostni pas/čelado (Da/Ne)  
32. **VozniskiStazVLetih**: vozniški staž osebe za kategorijo, ki jo potrebuje glede na vrsto udeleženca v prometu (LL)
33. **VozniskiStazVMesecih**: vozniški staž osebe za kategorijo, ki jo potrebuje glede na vrsto udeleženca v prometu (MM)
34. **VrednostAlkotesta**: vrednost alkotesta za osebo, če je bil opravljen (n.nn v enoti mg alkohola/liter izdihanega zraka (mg/l))
35. **VrednostStrokovnegaPregleda**: vrednost strokovnega pregleda za osebo, če je bil odrejen in so rezultati že znani (n.nn v enoti g alkohola/kg krvi (g/kg))

## Priprava podatkov

In [ ]:
combined_data = load_data()
combined_data


## Analiza podtakov

In [ ]:
classification_colors = {
    'Z MATERIALNO ŠKODO': '#377eb8',  
    'Z LAŽJO TELESNO POŠKODBO': '#4daf4a', 
    'S HUDO TELESNO POŠKODBO': '#ff7f00',  
    'S SMRTNIM IZIDOM': '#e41a1c'  
}
stacked_city_data = stack_data(combined_data, "UpravnaEnotaStoritve").head(10)

stacked_city_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_city_data.columns],
    figsize=(10, 5)
)

plt.title('Mesta z največ nesrečami', fontsize=14)
plt.xlabel('Število nesreč', fontsize=12)
plt.ylabel('Mesto', fontsize=12)
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.savefig('./slike/nesrecePoMestih.png')
plt.show()

In [ ]:
promet = pd.read_csv("podatki/vsota_vozil_po_upravnih_enotah.csv")
unique_accidents_2023 = combined_data[combined_data['DatumPN'].str.endswith('.2023')].groupby('ZaporednaStevilkaPN').first().reset_index()

nesrece_po_enoti = unique_accidents_2023.groupby('UpravnaEnotaStoritve').size().reset_index(name='Število nesreč')

promet = promet.dropna(subset=['Upravna enota'])
promet = promet[pd.to_numeric(promet['Vsota vseh vozil (PLDP)'], errors='coerce').notnull()]
promet['Vsota vseh vozil (PLDP)'] = promet['Vsota vseh vozil (PLDP)'].astype(int)

df = pd.merge(nesrece_po_enoti, promet, left_on='UpravnaEnotaStoritve', right_on='Upravna enota', how='inner')

df['Nesreč na 100k vozil'] = df['Število nesreč'] / df['Vsota vseh vozil (PLDP)'] * 100_000

df_filtered = df[~df['UpravnaEnotaStoritve'].str.upper().isin(['RUŠE', 'HRASTNIK', 'PIRAN'])]

df_top = df_filtered.sort_values('Nesreč na 100k vozil', ascending=False).head(10)

plt.figure(figsize=(12, 6))
plt.barh(df_top['UpravnaEnotaStoritve'], df_top['Nesreč na 100k vozil'], color='blue')
plt.xlabel('Število nesreč na 100.000 vozil')
plt.ylabel('Upravna enota')
plt.title('Upravne enote z največ nesrečami v letu 2023')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
filtered_data = combined_data[combined_data['UporabaVarnostnegaPasu'].notnull()]

stacked_data = filtered_data.pivot_table(
    index='UporabaVarnostnegaPasu',
    columns='KlasifikacijaNesrece',
    aggfunc='size',
    fill_value=0
)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
stacked_data = stacked_data[ordered_columns]

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Nesreče po uporabi varnostnega pasu', fontsize=14)
plt.xlabel('Število nesreč', fontsize=12)
plt.ylabel('Uporaba varnostnega pasu', fontsize=12)
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.savefig('./slike/nesrecePoUporabiVarnostnegaPasu.png')
plt.show()


In [ ]:
filtered_data = combined_data[combined_data['UporabaVarnostnegaPasu'].notnull()]

injury_counts = filtered_data.pivot_table(
    index='UporabaVarnostnegaPasu',
    columns='KlasifikacijaNesrece',
    aggfunc='size',
    fill_value=0
)

injury_ratios = injury_counts.div(injury_counts.sum(axis=1), axis=0)

injury_ratios.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in injury_ratios.columns],
    figsize=(10, 5)
)

plt.title('Razmerje med poškodbami glede na uporabo varnostnega pasu', fontsize=14)
plt.xlabel('Razmerje', fontsize=12)
plt.ylabel('Uporaba varnostnega pasu', fontsize=12)
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.savefig('./slike/razmerjeVarnostniPas.png')
plt.show()



In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()
culprits = unique_accidents[unique_accidents['Povzrocitelj'] == 'POVZROČITELJ']

total_by_gender = combined_data['Spol'].value_counts()

culprits_by_gender = culprits['Spol'].value_counts()

ratios = culprits_by_gender / total_by_gender
ratios

In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()
culprits = unique_accidents[unique_accidents['Povzrocitelj'] == 'POVZROČITELJ']

gender_counts = culprits.groupby(['Spol', 'KlasifikacijaNesrece']).size().unstack(fill_value=0)
gender_counts = gender_counts.loc[gender_counts.sum(axis=1).nlargest(2).index]

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
gender_counts = gender_counts[ordered_columns]

gender_counts.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in gender_counts.columns],
    figsize=(9, 4)
)

plt.title('Povzročitelji nesreč po spolu')
plt.xlabel('Število nesreč')
plt.ylabel('Spol')
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('./slike/nesrecePoSpolu.png')
plt.show()


In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()
culprits = unique_accidents[unique_accidents['Povzrocitelj'] == 'POVZROČITELJ']

culprits = culprits[culprits['Starost'] > 0]

culprits['StarostInterval'] = pd.cut(culprits['Starost'], bins=range(0, 101, 10), right=False)

age_counts = culprits.groupby(['StarostInterval', 'KlasifikacijaNesrece'], observed=False).size().unstack(fill_value=0)

age_counts['Total'] = age_counts.sum(axis=1)
age_counts = age_counts.sort_values(by='Total', ascending=False).drop(columns=['Total'])

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
age_counts = age_counts[ordered_columns]

age_counts.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in age_counts.columns],
    figsize=(9, 4)
)

plt.title('Povzročitelji nesreč po starostnih intervalih')
plt.xlabel('Število nesreč')
plt.ylabel('Starostni interval')
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('./slike/nesrecePoStarosti.png')
plt.show()


In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()
culprits = unique_accidents[unique_accidents['Povzrocitelj'] == 'POVZROČITELJ']

mladoletni_culprits = culprits[(culprits['Starost'] > 0) & (culprits['Starost'] < 18)]

mladoletni_counts = mladoletni_culprits.groupby(['VrstaUdelezenca', 'KlasifikacijaNesrece']).size().unstack(fill_value=0)

mladoletni_counts['Total'] = mladoletni_counts.sum(axis=1)
mladoletni_counts = mladoletni_counts.sort_values(by='Total', ascending=False).drop(columns=['Total'])

mladoletni_counts = mladoletni_counts.head(6)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
mladoletni_counts = mladoletni_counts[ordered_columns]

mladoletni_counts.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in mladoletni_counts.columns],
    figsize=(9, 4)
)

plt.title('Mladoletni povzročitelji nesreč po vrsti udeleženca')
plt.xlabel('Število nesreč')
plt.ylabel('Vrsta udeleženca')
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('./slike/nesreceMladoletnih.png')
plt.show()


In [ ]:
stacked_data = stack_data(combined_data, "VzrokNesrece")

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)


plt.title('Nesreče po vzrokih', fontsize=14)
plt.xlabel('Število nesreč', fontsize=12)
plt.ylabel('Vzrok nesreče', fontsize=12)
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.savefig('./slike/vzrokiNesrec.png')
plt.show()


In [ ]:
stacked_data = stack_data(combined_data, "VremenskeOkoliscine")

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Vremenske razmere pri nesreči', fontsize=14)
plt.xlabel('Število nesreč', fontsize=12)
plt.ylabel('Vzrok nesreče', fontsize=12)
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.savefig('./slike/vremeNesrec.png')
plt.show()

In [ ]:
filtered_data = combined_data[pd.to_datetime(combined_data['DatumPN'], format='%d.%m.%Y').dt.year >= 2013]

unique_accidents = filtered_data.groupby('ZaporednaStevilkaPN').first()

transformer = Transformer.from_crs("EPSG:3794", "EPSG:4326", always_xy=True)
unique_accidents['Lon'], unique_accidents['Lat'] = zip(*unique_accidents.apply(
    lambda row: transformer.transform(row['GeoKoordinataY'], row['GeoKoordinataX']), axis=1
))

valid_coordinates = unique_accidents[['Lat', 'Lon', 'KlasifikacijaNesrece']].dropna()
valid_coordinates = valid_coordinates[valid_coordinates['Lat'] >= 45]

gdf = gpd.GeoDataFrame(
    valid_coordinates,
    geometry=gpd.points_from_xy(valid_coordinates['Lon'], valid_coordinates['Lat']),
    crs='EPSG:4326'
)

gdf = gdf.to_crs(epsg=3857)

for classification, color in classification_colors.items():
    fig, ax = plt.subplots(figsize=(16, 9))
    subset_gdf = gdf[gdf['KlasifikacijaNesrece'] == classification]
    subset_gdf.plot(ax=ax, markersize=5, color=color, alpha=0.6, label=classification)
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    ax.set_title(f'Prometne nesreče: {classification} (od 2013 naprej)')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.legend(title="Klasifikacija nesreč")
    plt.tight_layout()
    plt.savefig(f'./slike/{classification.lower()}.png')
    plt.show()

    subset = valid_coordinates[valid_coordinates['KlasifikacijaNesrece'] == classification]
    heat_data = subset[['Lat', 'Lon']].dropna().values.tolist()
    m = folium.Map(location=[subset['Lat'].mean(), subset['Lon'].mean()], zoom_start=8)
    HeatMap(heat_data, radius=5, blur=6).add_to(m)
    print(f"Heatmap za klasifikacijo nesreče {classification.lower()}:")
    display(m)

fig, ax = plt.subplots(figsize=(16, 9))
for classification, color in classification_colors.items():
    subset_gdf = gdf[gdf['KlasifikacijaNesrece'] == classification]
    subset_gdf.plot(ax=ax, markersize=5, color=color, alpha=0.6, label=classification)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_title('Prometne nesreče po klasifikacijah (od 2013 naprej)')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(title="Klasifikacija nesreč")
plt.tight_layout()
plt.savefig('./slike/zemljevidNesrec.png')
plt.show()

print("Heatmap za vse klasifikacije nesreč:")
heat_data_all = valid_coordinates[['Lat', 'Lon']].dropna().values.tolist()
m_all = folium.Map(location=[valid_coordinates['Lat'].mean(), valid_coordinates['Lon'].mean()], zoom_start=8)
HeatMap(heat_data_all, radius=5, blur=6).add_to(m_all)
display(m_all)